This code is taken from https://github.com/arundasan91/MagnaTagATune. Credit to Arun Das.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import os
import shutil
import librosa

# Import MatPlotLib Package
import matplotlib.pyplot as plt

# Set number of columns to show in the notebook
pd.set_option('display.max_columns', 200)
# Set number of rows to show in the notebook
pd.set_option('display.max_rows', 50) 
# Make the graphs a bit prettier
plt.style.use('ggplot')

# Display pictures within the notebook itself
%matplotlib inline

In [ ]:
# Read the annotations file
newdata = pd.read_csv('annotations_final.csv', sep="\t")

In [ ]:
# Display the top 5 rows
newdata.head(5)

In [ ]:
# Get to know the data better
newdata.info()

In [ ]:
# What colums are there ?
newdata.columns

In [ ]:
# Extract the clip_id and mp3_path
newdata[["clip_id", "mp3_path"]]

In [ ]:
# Previous command extracted it as a Dataframe. We need it as a matrix to do analyics on. 
# Extract clip_id and mp3_path as a matrix.
clip_id, mp3_path = newdata[["clip_id", "mp3_path"]].as_matrix()[:,0], newdata[["clip_id", "mp3_path"]].as_matrix()[:,1]

In [ ]:
# Some of the tags in the dataset are really close to each other. Lets merge them together
synonyms = [['beat', 'beats'],
            ['chant', 'chanting'],
            ['choir', 'choral'],
            ['classical', 'clasical', 'classic'],
            ['drum', 'drums'],
            ['electro', 'electronic', 'electronica', 'electric'],
            ['fast', 'fast beat', 'quick'],
            ['female', 'female singer', 'female singing', 'female vocals', 'female vocal', 'female voice', 'woman', 'woman singing', 'women'],
            ['flute', 'flutes'],
            ['guitar', 'guitars'],
            ['hard', 'hard rock'],
            ['harpsichord', 'harpsicord'],
            ['heavy', 'heavy metal', 'metal'],
            ['horn', 'horns'],
            ['india', 'indian'],
            ['jazz', 'jazzy'],
            ['male', 'male singer', 'male vocal', 'male vocals', 'male voice', 'man', 'man singing', 'men'],
            ['no beat', 'no drums'],
            ['no singer', 'no singing', 'no vocal','no vocals', 'no voice', 'no voices', 'instrumental'],
            ['opera', 'operatic'],
            ['orchestra', 'orchestral'],
            ['quiet', 'silence'],
            ['singer', 'singing'],
            ['space', 'spacey'],
            ['string', 'strings'],
            ['synth', 'synthesizer'],
            ['violin', 'violins'],
            ['vocal', 'vocals', 'voice', 'voices'],
            ['strange', 'weird']]

In [ ]:
# Merge the synonyms and drop all other columns than the first one.
"""
Example:
Merge 'beat', 'beats' and save it to 'beat'.
Merge 'classical', 'clasical', 'classic' and save it to 'classical'.
"""
for synonym_list in synonyms:
    newdata[synonym_list[0]] = newdata[synonym_list].max(axis=1)
    newdata.drop(synonym_list[1:], axis=1, inplace=True)

In [ ]:
# Did it work ?
newdata.info()

In [ ]:
# Lets view it.
newdata.head()

In [ ]:
# Drop the mp3_path tag from the dataframe
newdata.drop('mp3_path', axis=1, inplace=True)
# Save the column names into a variable
data = newdata.sum(axis=0)

In [ ]:
# Find the distribution of tags.
data

In [ ]:
# Sort the column names.
data.sort_values(axis=0, inplace=True)

In [ ]:
# Find the top tags from the dataframe.
topindex, topvalues = list(data.index[84:]), data.values[84:]
del(topindex[-1])
topvalues = np.delete(topvalues, -1)

In [ ]:
# Get the top column names
topindex

In [ ]:
# Get only the top column values
topvalues

In [ ]:
# Get a list of columns to remove
rem_cols = data.index[:84]

In [ ]:
# Cross-check: How many columns are we removing ?
len(rem_cols)

In [ ]:
# Drop the columns that needs to be removed
newdata.drop(rem_cols, axis=1, inplace=True)

In [ ]:
newdata.info()

In [ ]:
# Create a backup of the dataframe
backup_newdata = newdata

In [ ]:
# Shuffle the dataframe
from sklearn.utils import shuffle
newdata = shuffle(newdata, random_state = 42)

In [ ]:
newdata.reset_index(drop=True)

In [ ]:
# One final check
newdata.info()

In [ ]:
# Let us save the final columns
final_columns_names = list(newdata.columns)

In [ ]:
# Do it only once to delete the clip_id column
del(final_columns_names[0])

In [ ]:
# Verified
final_columns_names

In [ ]:
# Create the file which is to be saved off (you could skip and apply similar steps in the previous dataframe)
# Here, binary 0's and 1's from each column is changed to 'False' and 'True' by using '==' operator on the dataframe.
final_matrix = pd.concat([newdata['clip_id'], newdata[final_columns_names]==1], axis=1)

In [ ]:
# Rename all the mp3 files to their clip_id and save it into one folder named 'dataset_clip_id_mp3' in the same folder.
# Get the current working directory
root = os.getcwd()
os.mkdir("D:/dataset_clip_id_mp3/", 0o755 )

# Iterate over the mp3 files, rename them to the clip_id and save it to another folder.
for id in range(25863):
    #print clip_id[id], mp3_path[id]
    src = "D:/mp3/" + mp3_path[id]
    dest = "D:/dataset_clip_id_mp3/" + str(clip_id[id]) + ".mp3"
    shutil.copy2(src,dest)
    #print src,dest

In [ ]:
def compute_melgram(audio_path):
    ''' Compute a mel-spectrogram and returns it in a shape of (1,1,96,1366), where
    96 == #mel-bins and 1366 == #time frame
    parameters
    ----------
    audio_path: path for the audio file.
                Any format supported by audioread will work.
    More info: http://librosa.github.io/librosa/generated/librosa.core.load.html#librosa.core.load
    '''

    # mel-spectrogram parameters
    SR = 12000
    N_FFT = 512
    N_MELS = 96
    HOP_LEN = 256
    DURA = 29.12  # to make it 1366 frame..

    src, sr = librosa.load(audio_path, sr=SR)  # whole signal
    n_sample = src.shape[0]
    n_sample_fit = int(DURA*SR)

    if n_sample < n_sample_fit:  # if too short
        src = np.hstack((src, np.zeros((int(DURA*SR) - n_sample,))))
    elif n_sample > n_sample_fit:  # if too long
        src = src[(n_sample-n_sample_fit)//2:(n_sample+n_sample_fit)//2]
    logam = librosa.logamplitude
    melgram = librosa.feature.melspectrogram
    ret = logam(melgram(y=src, sr=SR, hop_length=HOP_LEN,
                        n_fft=N_FFT, n_mels=N_MELS)**2,
                ref_power=1.0)
    ret = ret[np.newaxis, np.newaxis, :]
    return ret

In [ ]:
# Get the absolute path of all audio files and save it to audio_paths array
audio_paths = []
# Variable to save the mp3 files that don't work
files_that_dont_work=[]
os.chdir('D:/ex')
root = os.getcwd()
os.chdir('D:/dataset_clip_id_mp3/')
for audio_path in os.listdir('.'):
    #audio_paths.append(os.path.abspath(fname))
    if os.path.isfile('D:/dataset_clip_id_melgram/' + str(os.path.splitext(audio_path)[0]) + '.npy'):
        #print "existtt"
        continue
    else:
        if str(os.path.splitext(audio_path)[1]) == ".mp3":
            try:
                melgram = compute_melgram(os.path.abspath(audio_path))
                dest = 'D:/dataset_clip_id_melgram/' + str(os.path.splitext(audio_path)[0])
                np.save(dest, melgram)
            except EOFError:
                files_that_dont_work.append(audio_path)
                continue

In [ ]:
# Get a list of 
mp3_available = []
melgram_available = []
for mp3 in os.listdir('D:/dataset_clip_id_mp3/'):
     mp3_available.append(int(os.path.splitext(mp3)[0]))
        
for melgram in os.listdir('D:/dataset_clip_id_melgram/'):
     melgram_available.append(int(os.path.splitext(melgram)[0]))

In [ ]:
# The latest clip_id
new_clip_id = final_matrix['clip_id']

In [ ]:
# Let us see which all files have not been converted into melspectrograms.
set(list(new_clip_id)).difference(melgram_available)

In [ ]:
# Saw that these clips were extra 35644, 55753, 57881. Removing them.
final_matrix = final_matrix[final_matrix['clip_id']!= 35644]
final_matrix = final_matrix[final_matrix['clip_id']!= 55753]
final_matrix = final_matrix[final_matrix['clip_id']!= 57881]

In [ ]:
# Check again
final_matrix.info()

In [ ]:
# Save the matrix
final_matrix.to_pickle('D:/final_Dataframe.pkl')
# This is how you can load it afterwards. final_matrix = pd.read_pickle('final_Dataframe.pkl')

In [ ]:
# Seperate the training, test and validation dataframe.
training_with_clip = final_matrix[:19773]
validation_with_clip = final_matrix[19773:21294]
testing_with_clip = final_matrix[21294:]

In [ ]:
# Quick peek
training_with_clip

In [ ]:
# Quick peek
testing_with_clip

In [ ]:
# Quick peek
validation_with_clip

In [ ]:
# Extract the corresponding clip_id's
training_clip_id = training_with_clip['clip_id'].values
validation_clip_id = validation_with_clip['clip_id'].values
testing_clip_id = testing_with_clip['clip_id'].values

In [ ]:
# Check !
training_clip_id

In [ ]:
# Go to the directory you want to save the dataframe
os.chdir('D:/final_dataset')

In [ ]:
# Save the 'y' values.
np.save('train_y.npy', training_with_clip[final_columns_names].values)
np.save('valid_y.npy', validation_with_clip[final_columns_names].values)
np.save('test_y.npy', testing_with_clip[final_columns_names].values)

In [ ]:
# Save the 'x' clip_id's. We will make the numpy array using this.
np.savetxt('train_x_clip_id.txt', training_with_clip['clip_id'].values, fmt='%i')
np.savetxt('test_x_clip_id.txt', testing_with_clip['clip_id'].values, fmt='%i')
np.savetxt('valid_x_clip_id.txt', validation_with_clip['clip_id'].values, fmt='%i')

In [ ]:
# Now to combine the melgrams according to the clip_id.

# Variable to store melgrams.
train_x = np.zeros((0, 1, 96, 1366))
test_x = np.zeros((0, 1, 96, 1366))
valid_x = np.zeros((0, 1, 96, 1366))

In [ ]:
root = 'D:/'
os.chdir(root + "/dataset_clip_id_melgram/")
for i,valid_clip in enumerate(list(validation_clip_id)):
    if os.path.isfile(str(valid_clip) + '.npy'):
        #print i,valid_clip
        melgram = np.load(str(valid_clip) + '.npy')
        valid_x = np.concatenate((valid_x, melgram), axis=0)
os.chdir('D:/sets/')
np.save('valid_x.npy', valid_x)
print("Validation file created")

In [ ]:
root = 'D:/'
os.chdir(root + "/dataset_clip_id_melgram/")
for i,test_clip in enumerate(list(testing_clip_id)):
    if os.path.isfile(str(test_clip) + '.npy'):
        #print i,test_clip
        melgram = np.load(str(test_clip) + '.npy')
        test_x = np.concatenate((test_x, melgram), axis=0)
os.chdir('D:/sets/')
np.save('test_x.npy', test_x)
print("Testing file created")

In [ ]:
root = 'D:/'
os.chdir(root + "/dataset_clip_id_melgram/")
for i,train_clip in enumerate(list(training_clip_id)):
    ##if os.path.isfile(str(train_clip) + '.npy'):
        ##print i,train_clip
    #melgram = compute_melgram(str(train_clip) + '.mp3')
    ##melgram = np.load(str(train_clip) + '.npy')
    if os.path.isfile(str(train_clip) + '.npy'):
        melgram = np.load(str(train_clip) + '.npy')
        train_x = np.concatenate((train_x, melgram), axis=0)
os.chdir('D:/sets/')
np.save('train_x.npy', train_x)
print("Training file created.")